#Notebook for Inference using BIKE


NOTE:
For now it only works with CPU so please use CPU runtime

#Installing Libraries

In [1]:

!pip install torchnet
!pip install ftfy
!pip install dotmap
!pip install decord

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 26.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for torchnet: filename=torchnet-0.0.4-py3-none-any.whl size=29727 sha256=caef4a238f7c30a4c918b13ee86fb9bfd59a3bbbc03b8b67a1df823ea4bb0ba6
  Stored in directory: /root/.cache/pip/wheels/f7/ae/94/9f5edd6871983f30967ad11d60ef434c3d1b007654de4c8065
  Created wheel for visdom: filename=visdom-0.2.4-py3-none-any.whl size=1408195 sha256=c55c9fbba1df0fefc6bbf690340accd91cd9b1ee73059d4bd6e5c91a34b6b80e
  Stored in directory: /root/.cache/pip/wheels/42/29/49/5bed207bac4578e4d2c0c5fc0226bfd33a7e2953ea56356855
Successfully built torchnet visdom
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.0/43.0 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.6/13.6 MB 88.9 MB/s eta 0:00:00


#Downloading Model

In [2]:
import gdown

# The shared link for the second file
file_url = 'https://drive.google.com/uc?id=1tALbViwYTKYShThXp8x8n8upXkDajqre'

# Local path where the file will be saved
output_path = 'ucf-vit-l-f16.pt'  # Replace 'your_second_file_name.ext' with your desired file name and extension

# Download the file
gdown.download(file_url, output_path, quiet=False)

Downloading...
From (original): https://drive.google.com/uc?id=1tALbViwYTKYShThXp8x8n8upXkDajqre
From (redirected): https://drive.google.com/uc?id=1tALbViwYTKYShThXp8x8n8upXkDajqre&confirm=t&uuid=63546ca7-69e3-45e3-88c6-3d529747571f
To: /content/ucf-vit-l-f16.pt
100%|██████████| 4.65G/4.65G [00:41<00:00, 113MB/s]


'ucf-vit-l-f16.pt'

#Cloning Repo

In [3]:
!git clone https://github.com/hamzakhalil798/BIKE.git

Cloning into 'BIKE'...
remote: Enumerating objects: 313, done.
remote: Counting objects: 100% (313/313), done.
remote: Compressing objects: 100% (207/207), done.
remote: Total 313 (delta 127), reused 204 (delta 67), pack-reused 0 (from 0)
Receiving objects: 100% (313/313), 9.22 MiB | 11.37 MiB/s, done.
Resolving deltas: 100% (127/127), done.


#Inference for single Video

In [4]:
import os
import cv2
from google.colab import files

# Create the necessary directories
os.makedirs("customVideos", exist_ok=True)
os.makedirs("customVideos/Test", exist_ok=True)
os.makedirs("customVideos-Processed", exist_ok=True)

# Function to upload files to customVideos/Test folder
def upload_videos_to_test_folder():
    uploaded = files.upload()  # Upload files using Colab's file upload widget
    for filename in uploaded.keys():
        # Move the uploaded file to the customVideos/Test directory
        os.rename(filename, os.path.join("customVideos/Test", filename))

# Call the function to upload files
upload_videos_to_test_folder()

def extract_frames_from_videos(root_folder, output_folder, annotations_file):
    with open(annotations_file, 'w') as f:
        # Iterate over each category folder in the root directory
        for category in os.listdir(root_folder):
            category_path = os.path.join(root_folder, category)

            if os.path.isdir(category_path):
                # Iterate over each video in the category folder
                for video_file in os.listdir(category_path):
                    video_path = os.path.join(category_path, video_file)

                    # Create the output directory for the video frames
                    output_dir = os.path.join(output_folder, category, os.path.splitext(video_file)[0])
                    os.makedirs(output_dir, exist_ok=True)

                    # Capture the video
                    video_capture = cv2.VideoCapture(video_path)
                    success, frame = video_capture.read()
                    frame_count = 0

                    # Extract frames from the video
                    while success:
                        # Write the frame to the output directory
                        frame_filename = os.path.join(output_dir, f"image_{frame_count:04d}.jpg")
                        cv2.imwrite(frame_filename, frame)

                        # Read the next frame
                        success, frame = video_capture.read()
                        frame_count += 1

                    # Release the video capture object
                    video_capture.release()

                    # Write the video name and frame count to the annotations file
                    videoname = os.path.splitext(video_file)[0]
                    annotation_line = f"Test/{videoname} {frame_count} 0\n"
                    f.write(annotation_line)

root_folder = "/content/customVideos"  # Root folder containing the videos
output_folder = "/content/customVideos-Processed"  # Output folder for processed images
annotations_file = "/content/annotations.txt"  # File to save the annotations
extract_frames_from_videos(root_folder, output_folder, annotations_file)


Saving v_ApplyEyeMakeup_g01_c01.avi to v_ApplyEyeMakeup_g01_c01.avi


In [5]:
cd /content/BIKE

/content/BIKE


In [6]:
!python /content/BIKE/inference.py --config /content/BIKE/configs/ucf101/ucf_infer.yaml  --weights /content/ucf-vit-l-f16.pt --dist_url None

Not using distributed mode
100%|███████████████████████████████████████| 890M/890M [00:10<00:00, 90.5MiB/s]
dropout used:[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
dropout used:[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
loading clip pretrained model!
layer=6
video number:1
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:92: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warni